<a href="https://colab.research.google.com/github/williamchi64/williamchi64/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E5%81%9A%E5%9C%96%E5%83%8F%E8%B3%87%E6%96%99%E5%88%86%E5%89%B2%E8%88%87%E6%A8%A1%E5%9E%8B%E6%B8%AC%E8%A9%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip install gdown

In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 706kB 16.7MB/s 


In [ ]:
!wget -P /content/datasets https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
!tar -xf /content/datasets/wiki_crop.tar

--2021-05-10 16:50:51--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 811315200 (774M) [application/x-tar]
Saving to: ‘/content/datasets/wiki_crop.tar’

wiki_crop.tar       100%[===================>] 773.73M  30.6MB/s    in 27s     

2021-05-10 16:51:18 (29.1 MB/s) - ‘/content/datasets/wiki_crop.tar’ saved [811315200/811315200]



In [ ]:
!wget -O /content/datasets/wiki_final.csv drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP&export=download

--2021-05-10 16:51:23--  http://drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP
Resolving drive.google.com (drive.google.com)... 142.250.65.78, 2607:f8b0:4004:832::200e
Connecting to drive.google.com (drive.google.com)|142.250.65.78|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP [following]
--2021-05-10 16:51:23--  https://drive.google.com/u/0/uc?id=1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP
Connecting to drive.google.com (drive.google.com)|142.250.65.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3t4vffp8f9o2hmdb6ccn7f7a0kr7g6d5/1620665475000/01254859074079537426/*/1PDzAXHQWbHWQ4UxPLY8F13eTICWeyJGP [following]
--2021-05-10 16:51:25--  https://doc-14-0s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3t4vffp8f

In [ ]:
# IMDE image 人臉資料集來源: https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/

import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os 
import numpy as np
from tqdm import tqdm
import glob
import random

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Dense, Dropout,
                    Activation, Flatten, Conv2D,GlobalAveragePooling2D,
                    MaxPooling2D)
from tensorflow.keras.callbacks import LearningRateScheduler,EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input

import tensorflow_addons as tfa

In [ ]:
wiki = pd.read_csv('/content/datasets/wiki_final.csv')
wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44183 entries, 0 to 44182
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   year          44183 non-null  int64  
 1   photo_takens  44183 non-null  int64  
 2   path          44183 non-null  object 
 3   gender        44183 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.3+ MB


In [ ]:
def imagepath(index):
  index = index
  # print(wiki.iloc[index,:])
  path = os.path.join('/content/wiki_crop', wiki.iloc[index,2])
  return path

In [ ]:
def imgprocess(size,columns,sample_num):
  imgs = list()
  lbls = list()
  count = len(wiki[columns].unique())
  size = size
  for i in tqdm((sample_num), position=0, leave=True):
    img = cv2.imread(imagepath(i))
    img = cv2.resize(img, size)
    imgs.append(img)
    lbl = int(wiki[columns][i])
    lbl_onehot = np.zeros(count,dtype=float)
    lbl_onehot[lbl] = 1
    lbls.append(lbl_onehot)
  imgs = np.array(imgs)
  lbls = np.array(lbls)
  return {'images':imgs,'labels':lbls}

In [ ]:
# data index for random choosing
index_female = wiki[wiki['gender']==0].index
index_male = wiki[wiki['gender']==1].index
random_female = random.sample(list(index_female), 2000)
random_male = random.sample(list(index_male), 2000)
random_sample = random_female + random_male

In [ ]:
data = imgprocess((299,299),'gender',random_sample)

100%|██████████| 4000/4000 [00:16<00:00, 248.29it/s]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data['images'],data['labels'],test_size=0.2, random_state=500)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.2, random_state=500)

In [ ]:
del data

In [ ]:
!mkdir model-old
!gdown -O /content/model-old/inceptionV3_241.h5  https://drive.google.com/uc?id=1sxas71-rtKfd3tovyYf1FdtHoL6IJ4Bq
!gdown -O /content/model-old/inceptionV3_216.h5 https://drive.google.com/uc?id=1P0oPF7J-3dpfmWWrIcTA-CKiaWy699kR
!gdown -O /content/model-old/inceptionV3_0.h5 https://drive.google.com/uc?id=150_X4rkaHUJdUPtlEiFIOMfIazFbyTiW
!gdown -O /content/model-old/inceptionV3_-47.h5 https://drive.google.com/uc?id=1rpNF7665obvwbA9pAdnx6pHHwNn-3CG4
!gdown -O /content/model-old/inceptionV3_-43.h5 https://drive.google.com/uc?id=1VXkaTvmRHLozLcndv7NIqjr2GTSoXQSo
!gdown -O /content/model-old/inceptionV3.h5 https://drive.google.com/uc?id=1Va4prDpkot0K3sntwUrjQDYLB3POP9i2
!gdown -O /content/model-old/inceptionV3_1.h5 https://drive.google.com/uc?id=1f19b-rSSnWfPonBTZS06NsZzHKQvMFSY
!gdown -O /content/model-old/inceptionresnetV2.h5 https://drive.google.com/uc?id=1zXw0907GLvXiHwXiLpgXrZsdoNUu87mI

Downloading...
From: https://drive.google.com/uc?id=1sxas71-rtKfd3tovyYf1FdtHoL6IJ4Bq
To: /content/model-old/inceptionV3_241.h5
199MB [00:04, 40.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1P0oPF7J-3dpfmWWrIcTA-CKiaWy699kR
To: /content/model-old/inceptionV3_216.h5
199MB [00:04, 46.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=150_X4rkaHUJdUPtlEiFIOMfIazFbyTiW
To: /content/model-old/inceptionV3_0.h5
140MB [00:04, 34.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rpNF7665obvwbA9pAdnx6pHHwNn-3CG4
To: /content/model-old/inceptionV3_-47.h5
140MB [00:02, 50.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VXkaTvmRHLozLcndv7NIqjr2GTSoXQSo
To: /content/model-old/inceptionV3_-43.h5
140MB [00:03, 46.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Va4prDpkot0K3sntwUrjQDYLB3POP9i2
To: /content/model-old/inceptionV3.h5
89.7MB [00:01, 49.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1f19b-rSSnWfPonBTZS06NsZzHKQvMFSY
To: /co

In [ ]:
def to1Dtensor(y_input):
  y_list = []
  y_onehot = np.round(y_input)
  n_class = len(y_onehot[:][0])
  for j in y_onehot:
    for i in range(n_class):
      if j[i] == 1:
        y_list.append(i)
  return y_list

In [ ]:
batch_size = 32

In [ ]:
h5list = ['inceptionV3','inceptionV3_-43','inceptionV3_-47','inceptionV3_0','inceptionV3_1','inceptionV3_216','inceptionV3_241']
y_test_1D = to1Dtensor(y_test)
for h5 in h5list:
  test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
  test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
  test = load_model(f'/content/model-old/{h5}.h5')
  loss,acc = test.evaluate_generator(test_data, verbose=1)
  print(f'{h5}_loss:{loss}_accuracy:{acc}')
  predictions = test.predict(x=preprocess_input(x_test), verbose=1)
  pred_1D = to1Dtensor(predictions)
  cm = tf.math.confusion_matrix(y_test_1D, pred_1D, num_classes=2, weights=None, dtype=tf.dtypes.int32, name=h5)
  print(f'{h5}_confusion_matrix:{cm}')
  metric = tfa.metrics.F1Score(num_classes=1,threshold=0.5)
  y_t = np.array(y_test_1D).reshape(len(y_test_1D),1)
  y_p = np.array(pred_1D).reshape(len(pred_1D),1)
  metric.update_state(y_t, y_p)
  result=metric.result()
  print(f'{h5}_F1_score:{result.numpy()}')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 146s 6s/step - loss: 0.3468 - accuracy: 0.8537
inceptionV3_loss:0.3468235731124878_accuracy:0.8537499904632568
25/25 [==============================] - 146s 6s/step
inceptionV3_confusion_matrix:[[306  80]
 [ 37 377]]
inceptionV3_F1_score:[0.86567163]
25/25 [==============================] - 147s 6s/step - loss: 0.7363 - accuracy: 0.8612
inceptionV3_-43_loss:0.7362817525863647_accuracy:0.8612499833106995
25/25 [==============================] - 146s 6s/step
inceptionV3_-43_confusion_matrix:[[296  90]
 [ 21 393]]
inceptionV3_-43_F1_score:[0.87625426]
25/25 [==============================] - 147s 6s/step - loss: 1.0174 - accuracy: 0.8562
inceptionV3_-47_loss:1.0174360275268555_accuracy:0.856249988079071
25/25 [==============================] - 146s 6s/step
inceptionV3_-47_confusion_matrix:[[279 107]
 [  8 406]]
inceptionV3_-47_F1_score:[0.8759439]
25/25 [==============================] - 147s 6s/step - loss: 0.9097 - accuracy: 0.8775
inceptionV3_0_

ValueError: ignored

In [ ]:
# for inceptionresnetv2
data_1 = imgprocess((150,150),'gender',random_sample)

100%|██████████| 4000/4000 [00:07<00:00, 554.69it/s]


In [ ]:
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(data_1['images'],data_1['labels'],test_size=0.2, random_state=500)
x_train_1, x_val_1, y_train_1, y_val_1 = train_test_split(x_train_1,y_train_1,test_size=0.2, random_state=500)

In [ ]:
del data_1

In [ ]:
h5 = 'inceptionresnetV2'
y_test_1D = to1Dtensor(y_test)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_data = test_datagen.flow(x_test,y_test,batch_size=batch_size)
test = load_model(f'/content/model-old/{h5}.h5')
loss,acc = test.evaluate_generator(test_data, verbose=1)
print(f'{h5}_loss:{loss}_accuracy:{acc}')
predictions = test.predict(x=preprocess_input(x_test), verbose=1)
pred_1D = to1Dtensor(predictions)
cm = tf.math.confusion_matrix(y_test_1D, pred_1D, num_classes=2, weights=None, dtype=tf.dtypes.int32, name=h5)
print(f'{h5}_confusion_matrix:{cm}')
metric = tfa.metrics.F1Score(num_classes=1,threshold=0.5)
y_t = np.array(y_test_1D).reshape(len(y_test_1D),1)
y_p = np.array(pred_1D).reshape(len(pred_1D),1)
metric.update_state(y_t, y_p)
result=metric.result()
print(f'{h5}_F1_score:{result.numpy()}')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


25/25 [==============================] - 68s 3s/step - loss: 0.5378 - accuracy: 0.8313
inceptionresnetV2_loss:0.5378101468086243_accuracy:0.831250011920929
25/25 [==============================] - 67s 3s/step
inceptionresnetV2_confusion_matrix:[[262 124]
 [ 11 403]]
inceptionresnetV2_F1_score:[0.8565356]


In [ ]:
# ensemble 
h5list = ['inceptionV3','inceptionV3_0','inceptionV3_1']
# 'inceptionresnetV2'
dict_pred = {}
for h5 in h5list:
  test = load_model(f'/content/model-old/{h5}.h5')
  predictions = test.predict(x=preprocess_input(x_test), verbose=1)
  dict_pred[h5]=predictions

25/25 [==============================] - 146s 6s/step


In [ ]:
h5 = 'inceptionresnetV2'
test = load_model(f'/content/model-old/{h5}.h5')
predictions = test.predict(x=preprocess_input(x_test_1), verbose=1)
dict_pred[h5]=predictions

25/25 [==============================] - 67s 3s/step


In [ ]:
dict_pred.keys()

dict_keys(['inceptionV3', 'inceptionV3_0', 'inceptionV3_1', 'inceptionresnetV2'])

In [ ]:
mean_pred_1 = (dict_pred['inceptionV3']+dict_pred['inceptionV3_0']+dict_pred['inceptionV3_1'])/3
mean_pred_2 = (dict_pred['inceptionV3']+dict_pred['inceptionV3_0']+dict_pred['inceptionV3_1']+dict_pred['inceptionresnetV2'])/4
mean_pred = (mean_pred_1+mean_pred_2)/2

In [ ]:
def meanmatric(mean_pred):
  y_test_1D = to1Dtensor(y_test)
  pred_1D = to1Dtensor(mean_pred)
  cm = tf.math.confusion_matrix(y_test_1D, pred_1D, num_classes=2, weights=None, dtype=tf.dtypes.int32)
  print(f'{h5}_confusion_matrix:{cm}')
  metric = tfa.metrics.F1Score(num_classes=1,threshold=0.5)
  y_t = np.array(y_test_1D).reshape(len(y_test_1D),1)
  y_p = np.array(pred_1D).reshape(len(pred_1D),1)
  metric.update_state(y_t, y_p)
  result=metric.result()
  print(f'{h5}_F1_score:{result.numpy()}')

In [ ]:
meanmatric(mean_pred_1)

inceptionresnetV2_confusion_matrix:[[335  51]
 [ 28 386]]
inceptionresnetV2_F1_score:[0.90716803]


In [ ]:
meanmatric(mean_pred_2)

inceptionresnetV2_confusion_matrix:[[328  58]
 [ 19 395]]
inceptionresnetV2_F1_score:[0.91118795]


In [ ]:
meanmatric(mean_pred)

inceptionresnetV2_confusion_matrix:[[331  55]
 [ 20 394]]
inceptionresnetV2_F1_score:[0.9130938]
